## A reference game where one scalar implicature could be calculated

In [1]:
import numpy as np
import pandas as pd

In [2]:
class RSA:
    """Implementation of the core Rational Speech Acts model.

    Parameters
    ----------
    lexicon : `np.array` or `pd.DataFrame`
        Messages along the rows, states along the columns.
    prior : array-like
        Same length as the number of colums in `lexicon`.
    costs : array-like
        Same length as the number of rows in `lexicon`.
    alpha : float
        The temperature parameter. Default: 1.0
    """
    def __init__(self, lexicon, prior, costs, alpha=1.0):
        self.lexicon = lexicon
        self.prior = np.array(prior)
        self.costs = np.array(costs)
        self.alpha = alpha

    def literal_listener(self):
        """Literal listener predictions, which corresponds intuitively
        to truth conditions with priors.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.

        """
        return rownorm(self.lexicon * self.prior)

    def speaker(self):
        """Returns a matrix of pragmatic speaker predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to states, the columns to states.
        """
        lit = self.literal_listener().T
        utilities = self.alpha * (safelog(lit) + self.costs)
        return rownorm(np.exp(utilities))

    def listener(self):
        """Returns a matrix of pragmatic listener predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.
        """
        spk = self.speaker().T
        return rownorm(spk * self.prior)


def rownorm(mat):
    """Row normalization of np.array or pd.DataFrame"""
    return (mat.T / mat.sum(axis=1)).T


def safelog(vals):
    """Silence distracting warnings about log(0)."""
    with np.errstate(divide='ignore'):
        return np.log(vals)


if __name__ == '__main__':
    """Examples"""

    from IPython.display import display


    def display_reference_game(mod):
        d = mod.lexicon.copy()
        d['costs'] = mod.costs
        d.loc['prior'] = list(mod.prior) + [""]
        d.loc['alpha'] = [mod.alpha] + [" "] * mod.lexicon.shape[1]
        display(d)


    # Core lexicon:
    msgs = ['hat', 'glasses', 'mustache']
    states = ['r1', 'r2', 'r3']
    lex = pd.DataFrame([
        [1.0, 1.0, 0.0],
        [0.0, 0.0, 1.0],
        [0.0, 1.0, 0.0]], index=msgs, columns=states)

In [3]:
    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0.0, 0.0, 0.0], alpha=1.0)

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,costs
hat,1.000000,1.0,0.0,0.0
glasses,0.000000,0.0,1.0,0.0
mustache,0.000000,1.0,0.0,0.0
prior,0.333333,0.333333,0.333333,
alpha,1.000000,,,



Literal listener


,r1,r2,r3
hat,0.5,0.5,0.0
glasses,0.0,0.0,1.0
mustache,0.0,1.0,0.0



Pragmatic speaker


,hat,glasses,mustache
r1,1.000000,0.0,0.000000
r2,0.333333,0.0,0.666667
r3,0.000000,1.0,0.000000



Pragmatic listener


,r1,r2,r3
hat,0.75,0.25,0.0
glasses,0.00,0.00,1.0
mustache,0.00,1.00,0.0


From the table of the pragmatic listener above we can derive a scalar implicature. In this game, utterances "hat, mustache" form a scale, where "mustache" is more informative than "hat" because it picks out a smaller set of objects than the "hat" does. The scalar implicature that could be calculated by the listener in this scenario: "the speaker said "hat", the pressures for the speaker are to be as truthful and as informative as they can, "mustache" is more informative than "hat", if the speaker chose to use the less informative alternative, then the more informative alternative must be false, hence the implicature can be derived "hat, but not mustache"". We see that this implicature can be derived from the table of the pragmatic listener above: the probability that the listener infers "hat, but not mustache" is significantly higher than chance (75%).

In [4]:
    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[0.1, 0.45, 0.45], costs=[0.0, 0.0, 0.0], alpha=1.0)

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,costs
hat,1.0,1.0,0.0,0.0
glasses,0.0,0.0,1.0,0.0
mustache,0.0,1.0,0.0,0.0
prior,0.1,0.45,0.45,
alpha,1.0,,,



Literal listener


,r1,r2,r3
hat,0.181818,0.818182,0.0
glasses,0.000000,0.000000,1.0
mustache,0.000000,1.000000,0.0



Pragmatic speaker


,hat,glasses,mustache
r1,1.00,0.0,0.00
r2,0.45,0.0,0.55
r3,0.00,1.0,0.00



Pragmatic listener


,r1,r2,r3
hat,0.330579,0.669421,0.0
glasses,0.000000,0.000000,1.0
mustache,0.000000,1.000000,0.0


In this scenario of the game, the implicature is not calculated by the pragmatic listener due to the disparency between the prior probabilities of r1 and r2 with r2 being a priori more likely to be picked out. 

## A reference game where two scalar implicatures could be calculated

1. Scenario 1: informativity scale with three elements, no distractors

In [30]:
class RSA:
    """Implementation of the core Rational Speech Acts model.

    Parameters
    ----------
    lexicon : `np.array` or `pd.DataFrame`
        Messages along the rows, states along the columns.
    prior : array-like
        Same length as the number of colums in `lexicon`.
    costs : array-like
        Same length as the number of rows in `lexicon`.
    alpha : float
        The temperature parameter. Default: 1.0
    """
    def __init__(self, lexicon, prior, costs, alpha=1.0):
        self.lexicon = lexicon
        self.prior = np.array(prior)
        self.costs = np.array(costs)
        self.alpha = alpha

    def literal_listener(self):
        """Literal listener predictions, which corresponds intuitively
        to truth conditions with priors.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.

        """
        return rownorm(self.lexicon * self.prior)

    def speaker(self):
        """Returns a matrix of pragmatic speaker predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to states, the columns to states.
        """
        lit = self.literal_listener().T
        utilities = self.alpha * (safelog(lit) + self.costs)
        return rownorm(np.exp(utilities))

    def listener(self):
        """Returns a matrix of pragmatic listener predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.
        """
        spk = self.speaker().T
        return rownorm(spk * self.prior)


def rownorm(mat):
    """Row normalization of np.array or pd.DataFrame"""
    return (mat.T / mat.sum(axis=1)).T


def safelog(vals):
    """Silence distracting warnings about log(0)."""
    with np.errstate(divide='ignore'):
        return np.log(vals)


if __name__ == '__main__':
    """Examples"""

    from IPython.display import display


    def display_reference_game(mod):
        d = mod.lexicon.copy()
        d['costs'] = mod.costs
        d.loc['prior'] = list(mod.prior) + [""]
        d.loc['alpha'] = [mod.alpha] + [" "] * mod.lexicon.shape[1]
        display(d)


    # Core lexicon:
    msgs = ['hat', 'glasses', 'mustache']
    states = ['r1', 'r2', 'r3']
    lex = pd.DataFrame([
        [1.0, 1.0, 1.0],
        [0.0, 0.0, 1.0],
        [0.0, 1.0, 1.0]], index=msgs, columns=states)

In [6]:
    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0.0, 0.0, 0.0], alpha=1.0)

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,costs
hat,1.000000,1.0,1.0,0.0
glasses,0.000000,0.0,1.0,0.0
mustache,0.000000,1.0,1.0,0.0
prior,0.333333,0.333333,0.333333,
alpha,1.000000,,,



Literal listener


,r1,r2,r3
hat,0.333333,0.333333,0.333333
glasses,0.000000,0.000000,1.000000
mustache,0.000000,0.500000,0.500000



Pragmatic speaker


,hat,glasses,mustache
r1,1.000000,0.000000,0.000000
r2,0.400000,0.000000,0.600000
r3,0.181818,0.545455,0.272727



Pragmatic listener


,r1,r2,r3
hat,0.632184,0.252874,0.114943
glasses,0.000000,0.000000,1.000000
mustache,0.000000,0.687500,0.312500


+ The informativity scale: \<hat, mustache, glasses\> with "glasses" being the most informative.
+ Scalar implicature 1: "hat, but not mustache/glasses"
+ Scalar implicature 2: "mustache, but not glasses"
+ From the table for pragmatic listener above we see that RSA predicts that both imlicatures will be calculated, the strength of the first implicature is lower than the strength of the second implicature, and lower than the strength of the implicature in the corresponding one scalar implicature scenario

From the game with one scalar implicature we know that the higher the alpha, the stronger the implicature (in one scalar implicature scenario, alpha = 4 yields a 0,94 probability that the implicature will be calculated). Let's set alpha to four and see what result it yields in this scenario.

In [25]:
    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0.0, 0.0, 0.0], alpha=4.0)

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,costs
hat,1.000000,1.0,1.0,0.0
glasses,0.000000,0.0,1.0,0.0
mustache,0.000000,1.0,1.0,0.0
prior,0.333333,0.333333,0.333333,
alpha,4.000000,,,



Literal listener


,r1,r2,r3
hat,0.333333,0.333333,0.333333
glasses,0.000000,0.000000,1.000000
mustache,0.000000,0.500000,0.500000



Pragmatic speaker


,hat,glasses,mustache
r1,1.000000,0.000000,0.000000
r2,0.164948,0.000000,0.835052
r3,0.011486,0.930366,0.058148



Pragmatic listener


,r1,r2,r3
hat,0.850026,0.140210,0.009763
glasses,0.000000,0.000000,1.000000
mustache,0.000000,0.934899,0.065101


As we can see from the table above, alpha of four strengthens both implicatures.

Now let's manipulate the prior probabilities by lowering significantly the priors associated with less informative alternatives. As we know from the one implicature scenario, this should result in the implicature not being calculated.

In [26]:
    print("="*70 + "\nUneven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[0.15, 0.15, 0.7], costs=[0.0, 0.0, 0.0], alpha=1.0)

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Uneven priors and all-0 message costs



,r1,r2,r3,costs
hat,1.00,1.0,1.0,0.0
glasses,0.00,0.0,1.0,0.0
mustache,0.00,1.0,1.0,0.0
prior,0.15,0.15,0.7,
alpha,1.00,,,



Literal listener


,r1,r2,r3
hat,0.15,0.150000,0.700000
glasses,0.00,0.000000,1.000000
mustache,0.00,0.176471,0.823529



Pragmatic speaker


,hat,glasses,mustache
r1,1.000000,0.00000,0.000000
r2,0.459459,0.00000,0.540541
r3,0.277389,0.39627,0.326340



Pragmatic listener


,r1,r2,r3
hat,0.363116,0.166837,0.470047
glasses,0.000000,0.000000,1.000000
mustache,0.000000,0.261958,0.738042


As we can see from the table above, both implicatures are not calculated.

Let's now additionally to changed priors set alpha to four, which should strengthen the implicatures.

In [27]:
    print("="*70 + "\nUneven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[0.15, 0.15, 0.7], costs=[0.0, 0.0, 0.0], alpha=4.0)

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Uneven priors and all-0 message costs



,r1,r2,r3,costs
hat,1.00,1.0,1.0,0.0
glasses,0.00,0.0,1.0,0.0
mustache,0.00,1.0,1.0,0.0
prior,0.15,0.15,0.7,
alpha,4.00,,,



Literal listener


,r1,r2,r3
hat,0.15,0.150000,0.700000
glasses,0.00,0.000000,1.000000
mustache,0.00,0.176471,0.823529



Pragmatic speaker


,hat,glasses,mustache
r1,1.000000,0.000000,0.000000
r2,0.342972,0.000000,0.657028
r3,0.141231,0.588216,0.270554



Pragmatic listener


,r1,r2,r3
hat,0.499488,0.171311,0.329201
glasses,0.000000,0.000000,1.000000
mustache,0.000000,0.342271,0.657729


We see that now the first implicature is calculated (even though it is very weak), the second implicature is not.

I will not manipulate the utterance cost here, moving on to the next scenario.

2. Scenario 2: two scales

In [36]:
class RSA:
    """Implementation of the core Rational Speech Acts model.

    Parameters
    ----------
    lexicon : `np.array` or `pd.DataFrame`
        Messages along the rows, states along the columns.
    prior : array-like
        Same length as the number of colums in `lexicon`.
    costs : array-like
        Same length as the number of rows in `lexicon`.
    alpha : float
        The temperature parameter. Default: 1.0
    """
    def __init__(self, lexicon, prior, costs, alpha=1.0):
        self.lexicon = lexicon
        self.prior = np.array(prior)
        self.costs = np.array(costs)
        self.alpha = alpha

    def literal_listener(self):
        """Literal listener predictions, which corresponds intuitively
        to truth conditions with priors.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.

        """
        return rownorm(self.lexicon * self.prior)

    def speaker(self):
        """Returns a matrix of pragmatic speaker predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to states, the columns to states.
        """
        lit = self.literal_listener().T
        utilities = self.alpha * (safelog(lit) + self.costs)
        return rownorm(np.exp(utilities))

    def listener(self):
        """Returns a matrix of pragmatic listener predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.
        """
        spk = self.speaker().T
        return rownorm(spk * self.prior)


def rownorm(mat):
    """Row normalization of np.array or pd.DataFrame"""
    return (mat.T / mat.sum(axis=1)).T


def safelog(vals):
    """Silence distracting warnings about log(0)."""
    with np.errstate(divide='ignore'):
        return np.log(vals)


if __name__ == '__main__':
    """Examples"""

    from IPython.display import display


    def display_reference_game(mod):
        d = mod.lexicon.copy()
        d['costs'] = mod.costs
        d.loc['prior'] = list(mod.prior) + [""]
        d.loc['alpha'] = [mod.alpha] + [" "] * mod.lexicon.shape[1]
        display(d)


    # Core lexicon:
    msgs = ['hat', 'glasses', 'mustache']
    states = ['r1', 'r2', 'r3']
    lex = pd.DataFrame([
        [1.0, 1.0, 0.0],
        [0.0, 0.0, 1.0],
        [0.0, 1.0, 1.0]], index=msgs, columns=states)

In [37]:
    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0.0, 0.0, 0.0], alpha=1.0)

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,costs
hat,1.000000,1.0,0.0,0.0
glasses,0.000000,0.0,1.0,0.0
mustache,0.000000,1.0,1.0,0.0
prior,0.333333,0.333333,0.333333,
alpha,1.000000,,,



Literal listener


,r1,r2,r3
hat,0.5,0.5,0.0
glasses,0.0,0.0,1.0
mustache,0.0,0.5,0.5



Pragmatic speaker


,hat,glasses,mustache
r1,1.0,0.000000,0.000000
r2,0.5,0.000000,0.500000
r3,0.0,0.666667,0.333333



Pragmatic listener


,r1,r2,r3
hat,0.666667,0.333333,0.0
glasses,0.000000,0.000000,1.0
mustache,0.000000,0.600000,0.4


+ The informativity scale 1: \<hat, mustache\> with "mustache" being the more informative alternative.
+ The informativity scale 2: \<mustache, glasses\> with "glasses" being the more informative alternative.
+ Scalar implicature 1: "hat, but not mustache"
+ Scalar implicature 2: "mustache, but not glasses"
+ From the table for pragmatic listener above we see that the RSA predicts that both imlicatures will be calculated, the strength of the first implicature is higher than the strength of the second implicature, but lower than the strength of the implicature in the corresponding one scalar implicature scenario

First let's set alpha to four. Our hypothesis based on the discussion above and in the lecture is that this will strengthen the implicatures.

In [38]:
    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0.0, 0.0, 0.0], alpha=4.0)

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,costs
hat,1.000000,1.0,0.0,0.0
glasses,0.000000,0.0,1.0,0.0
mustache,0.000000,1.0,1.0,0.0
prior,0.333333,0.333333,0.333333,
alpha,4.000000,,,



Literal listener


,r1,r2,r3
hat,0.5,0.5,0.0
glasses,0.0,0.0,1.0
mustache,0.0,0.5,0.5



Pragmatic speaker


,hat,glasses,mustache
r1,1.0,0.000000,0.000000
r2,0.5,0.000000,0.500000
r3,0.0,0.941176,0.058824



Pragmatic listener


,r1,r2,r3
hat,0.666667,0.333333,0.000000
glasses,0.000000,0.000000,1.000000
mustache,0.000000,0.894737,0.105263


The hypothesis is born out, but note that the second implicature is benefitting from this manipulation much more than the first.

Now let's change the priors, lowering the probability of words/objects associated with less informative alternatives.

In [39]:
    print("="*70 + "\nUneven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[0.15, 0.15, 0.7], costs=[0.0, 0.0, 0.0], alpha=1.0)

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Uneven priors and all-0 message costs



,r1,r2,r3,costs
hat,1.00,1.0,0.0,0.0
glasses,0.00,0.0,1.0,0.0
mustache,0.00,1.0,1.0,0.0
prior,0.15,0.15,0.7,
alpha,1.00,,,



Literal listener


,r1,r2,r3
hat,0.5,0.500000,0.000000
glasses,0.0,0.000000,1.000000
mustache,0.0,0.176471,0.823529



Pragmatic speaker


,hat,glasses,mustache
r1,1.00000,0.000000,0.000000
r2,0.73913,0.000000,0.260870
r3,0.00000,0.548387,0.451613



Pragmatic listener


,r1,r2,r3
hat,0.575,0.425000,0.000000
glasses,0.000,0.000000,1.000000
mustache,0.000,0.110146,0.889854


Both implicatures are not computed. 

Now let's experiment with uneven priors and alpha of four.

In [40]:
    print("="*70 + "\nUneven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[0.15, 0.15, 0.7], costs=[0.0, 0.0, 0.0], alpha=4.0)

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Uneven priors and all-0 message costs



,r1,r2,r3,costs
hat,1.00,1.0,0.0,0.0
glasses,0.00,0.0,1.0,0.0
mustache,0.00,1.0,1.0,0.0
prior,0.15,0.15,0.7,
alpha,4.00,,,



Literal listener


,r1,r2,r3
hat,0.5,0.500000,0.000000
glasses,0.0,0.000000,1.000000
mustache,0.0,0.176471,0.823529



Pragmatic speaker


,hat,glasses,mustache
r1,1.00000,0.000000,0.000000
r2,0.98472,0.000000,0.015280
r3,0.00000,0.684952,0.315048



Pragmatic listener


,r1,r2,r3
hat,0.503849,0.496151,0.000000
glasses,0.000000,0.000000,1.000000
mustache,0.000000,0.010286,0.989714


Almost no effect on the first implicature (still not calculated). The second implicature is not calculated either.

Now let's manipulate the costs.

In [44]:
    print("="*70 + "\nEven priors and uneven message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0.0, -6, -6], alpha=1.0)

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and uneven message costs



,r1,r2,r3,costs
hat,1.000000,1.0,0.0,0.0
glasses,0.000000,0.0,1.0,-6.0
mustache,0.000000,1.0,1.0,-6.0
prior,0.333333,0.333333,0.333333,
alpha,1.000000,,,



Literal listener


,r1,r2,r3
hat,0.5,0.5,0.0
glasses,0.0,0.0,1.0
mustache,0.0,0.5,0.5



Pragmatic speaker


,hat,glasses,mustache
r1,1.000000,0.000000,0.000000
r2,0.997527,0.000000,0.002473
r3,0.000000,0.666667,0.333333



Pragmatic listener


,r1,r2,r3
hat,0.500619,0.499381,0.000000
glasses,0.000000,0.000000,1.000000
mustache,0.000000,0.007363,0.992637


Making costly to utter more informative utterances results in the lack of implicatures.

Combining all three manipulations: uneven priors, uneven costs, alpha of four.

In [46]:
    print("="*70 + "\nUneven priors and uneven message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[0.15, 0.7, 0.15], costs=[0.0, -3, -3], alpha=4.0)

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Uneven priors and uneven message costs



,r1,r2,r3,costs
hat,1.00,1.0,0.0,0.0
glasses,0.00,0.0,1.0,-3.0
mustache,0.00,1.0,1.0,-3.0
prior,0.15,0.7,0.15,
alpha,4.00,,,



Literal listener


,r1,r2,r3
hat,0.176471,0.823529,0.000000
glasses,0.000000,0.000000,1.000000
mustache,0.000000,0.823529,0.176471



Pragmatic speaker


,hat,glasses,mustache
r1,1.000000,0.000000,0.000000
r2,0.999994,0.000000,0.000006
r3,0.000000,0.999031,0.000969



Pragmatic listener


,r1,r2,r3
hat,0.176471,0.823529,0.000000
glasses,0.000000,0.000000,1.000000
mustache,0.000000,0.028743,0.971257


Higher alpha strengthens the implicatures, but cannot overpower the costs and the priors. The implicature calculation is highly sensitive to a particular scenario of the game: the number of worlds/states/objects, the number of utterances, the number and makeup of scales.